<a href="https://colab.research.google.com/github/CE334/CE334_230445_Harsh_gupta/blob/main/CE334_LAB1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [102]:
import numpy as np

## Defining Constants

In [103]:
# WGS84 Constants
a = 6378137.0  # Semi-major axis (in meters)
f_inv = 298.257223563  # Inverse flattening
f = 1 / f_inv
e2 = 2 * f - f**2  # Squared eccentricity

# Kanpur Coordinates (In Degrees)
lat_Kanpur_deg = 26.449923
lon_Kanpur_deg =  80.331871
h_Kanpur = 126.630 # in meters

# Hometown(Narora, U.P) Coordinates (In Degrees)
lat_home_deg = 28.1968
lon_home_deg = 78.3814
h_home = 174.0 # in meters

## Answer 1

### Function for converting geodetic to cartesian

In [104]:
def geodetic_to_cartesian_p(phi_deg, lam_deg, h, a, e2):
    # Convert degrees to radians
    phi = np.radians(phi_deg)
    lam = np.radians(lam_deg)

    # Radius of curvature in prime vertical
    N = a / np.sqrt(1 - e2 * (np.sin(phi)**2))

    # Cartesian coordinates
    x = (N + h) * np.cos(phi) * np.cos(lam)
    y = (N + h) * np.cos(phi) * np.sin(lam)
    z = ((1 - e2) * N + h) * np.sin(phi)
    return x, y, z

### Finding cartesian coordinates of hometown

In [105]:
# Finding Cartesian coordinates of hometown
x_h, y_h, z_h = geodetic_to_cartesian_p(lat_home_deg, lon_home_deg, h_home, a, e2)

print('Hometown\'s Cartesian Coordinates:')
print(f'x: {x_h} m')
print(f'y: {y_h} m')
print(f'z: {z_h} m')

Hometown's Cartesian Coordinates:
x: 1132973.0845224194 m
y: 5510332.212868267 m
z: 2995826.5869421256 m


### Function for converting cartesian to geodetic

In [106]:
def cartesian_to_geodetic_p(x, y, z, a, e2, precision=1e-8):
    # Longitude calculation
    lam = np.arctan2(y, x)

    p = np.sqrt(x**2 + y**2)
    # Initial latitude
    phi_initial = np.arctan2(z / p, 1 - e2)

    # Iterative process
    while True:
        N = a / np.sqrt(1 - e2 * np.sin(phi_initial)**2)
        h = (p / np.cos(phi_initial)) - N

        # Improved latitude
        new_phi = np.arctan2(z / p, 1 - (e2 * N / (N + h)))

        if abs(new_phi - phi_initial) < precision:
            break
        phi_initial = new_phi

    return np.degrees(phi_initial), np.degrees(lam), h

## Answer 2

### Function for converting spherical to cartesian

In [107]:
def spherical_to_cartesian_p(r, phi_deg, lam_deg):
    # Convert degrees to radians
    phi = np.radians(phi_deg)
    lam = np.radians(lam_deg)

    x = r * np.cos(phi) * np.cos(lam)
    y = r * np.cos(phi) * np.sin(lam)
    z = r * np.sin(phi)

    return x, y, z

### Function for converting cartesian to spherical

In [108]:
def cartesian_to_spherical_p(x, y, z):
    # Radius of curvature
    r = np.sqrt(x**2 + y**2 + z**2)

    # Handling the origin case to avoid division by zero
    if r == 0:
        return 0.0, 0.0, 0.0

    phi_rad = np.asin(z / r)
    # Using atan2 for correct quadrant (-180 to 180 degrees)
    lam_rad = np.atan2(y, x)

    # Convert radians back to degrees
    phi_deg = np.degrees(phi_rad)
    lam_deg = np.degrees(lam_rad)

    return r, phi_deg, lam_deg

## Answer 3

### Function for converting geodetic to spherical

In [109]:
def geodetic_to_spherical(phi_g, lam_g, h_g, a, f):
    e2 = 2*f - f**2

    # Step 1: Geodetic to Cartesian
    x, y, z = geodetic_to_cartesian_p(phi_g, lam_g, h_g, a, e2)

    # Step 2: Cartesian to Spherical
    r, phi_s, lam_s = cartesian_to_spherical_p(x, y, z)

    return r, phi_s, lam_s

## Answer 4

### Function for converting geodetic to topocentric

In [110]:
def geodetic_to_topocentric(phi_target_deg, lam_target_deg, h_target, phi_ref_deg, lam_ref_deg, h_ref, a, f_inv):
    f = 1 / f_inv
    e2 = 2*f - f**2

    # Converting degrees to radians
    phi_ref = np.radians(phi_ref_deg)
    lam_ref = np.radians(lam_ref_deg)

    # Finding Cartesian coordinates for both points
    x_t, y_t, z_t = geodetic_to_cartesian_p(phi_target_deg, lam_target_deg, h_target, a, e2)
    x_r, y_r, z_r = geodetic_to_cartesian_p(phi_ref_deg, lam_ref_deg, h_ref, a, e2)

    # Displacement vector in ECEF
    dx, dy, dz = x_t - x_r, y_t - y_r, z_t - z_r

    # Transformation matrix
    matrix = np.array([
        [-np.sin(lam_ref), np.cos(lam_ref), 0],
        [-np.cos(lam_ref)*np.sin(phi_ref), -np.sin(lam_ref)*np.sin(phi_ref), np.cos(phi_ref)],
        [np.cos(lam_ref)*np.cos(phi_ref), np.sin(lam_ref)*np.cos(phi_ref), np.sin(phi_ref)]
    ])

    # Resulting ENU coordinates
    enu = np.dot(matrix, np.array([dx, dy, dz]))
    east, north, up = enu

    return east, north, up

### Finding topocentric coordinates of satellite

In [111]:
# Satellite Coordinates
satellite_height = 800000 # in meters
h_satellite = h_Kanpur + satellite_height # in meters

east, north, up = geodetic_to_topocentric(lat_Kanpur_deg, lon_Kanpur_deg, h_satellite, lat_home_deg, lon_home_deg, h_home, a, f_inv)

print('Topocentric Coordinates of Satellite:')
print(f'east: {east} m')
print(f'north: {north} m')
print(f'up: {up} m')

Topocentric Coordinates of Satellite:
east: 218871.4143207077 m
north: -216169.79289630952 m
up: 793346.6388237093 m


## Bonus Question

### Function for converting topocentric to geodetic

In [112]:
def topocentric_to_geodetic(de, dn, du, phi_ref_deg, lam_ref_deg, h_ref, a, e2):

    # Converting degrees to radians
    phi_ref = np.radians(phi_ref_deg)
    lam_ref = np.radians(lam_ref_deg)

    # Transformation matrix (Transpose of the ENU matrix)
    matrix_inv = np.array([
        [-np.sin(lam_ref), -np.cos(lam_ref)*np.sin(phi_ref), np.cos(lam_ref)*np.cos(phi_ref)],
        [np.cos(lam_ref), -np.sin(lam_ref)*np.sin(phi_ref), np.sin(lam_ref)*np.cos(phi_ref)],
        [0, np.cos(phi_ref), np.sin(phi_ref)]
    ])

    # ECEF displacement
    dx, dy, dz = np.dot(matrix_inv, np.array([de, dn, du]))

    # Reference Cartesian
    x_r, y_r, z_r = geodetic_to_cartesian_p(phi_ref_deg, lam_ref_deg, h_ref, a, e2)

    # Target Cartesian
    x_t, y_t, z_t = x_r + dx, y_r + dy, z_r + dz

    # Convert back to Geodetic
    return cartesian_to_geodetic_p(x_t, y_t, z_t, a, e2)

### Finding geodetic coordinates of aircraft

In [113]:
# Aircraft Coordinates
aircraft_up = 6000 # in meters
aircraft_east = -800 # in meters
aircraft_north = 900 # in meters

phi, lam, h = topocentric_to_geodetic(aircraft_east, aircraft_north, aircraft_up, lat_home_deg, lon_home_deg, h_home, a, e2)

print('Geodetic Coordinates of Aircraft:')
print(f'phi: {phi}°')
print(f'lambda: {lam}°')
print(f'h: {h} m')

Geodetic Coordinates of Aircraft:
phi: 28.204912716627987°
lambda: 78.37325917471215°
h: 6174.099972316064 m
